## 필요성
엑셀의 수식/서식/메모를 변경하지 않고 셀 값들만 변경할려면 엑셀객체에 api 접근하는 방법이 이용된다 합니다.
가용한 방법들은 아래와 같습니다.

+----------------------+-------------------+---------------------+------------------------------------------------+
| 구분                 | 라이브러리 / 기술 | 지원 운영체제       | 설명                                           |
+======================+===================+=====================+================================================+
| **VBA**              | 기본 매크로       | **Windows / macOS** | Excel 내장 스크립트 언어.                      |
|                      |                   |                     |                                                |
|                      |                   |                     | Excel 앱에서 직접 실행되며 모든 요소 완전 보존 |
|                      |                   |                     |                                                |
|                      |                   |                     | macOS에서 일부기능 제한적                      |
+----------------------+-------------------+---------------------+------------------------------------------------+
| **Python + pywin32** | `win32com.client` | **Windows 전용**    | Excel COM API 직접 호출.                       |
|                      |                   |                     |                                                |
|                      |                   |                     | VBA와 동일 수준의 제어 가능                    |
+----------------------+-------------------+---------------------+------------------------------------------------+
| **Python + xlwings** | `xlwings`         | **Windows / macOS** | COM(Windows)과 AppleScript(macOS) 기반의       |
|                      |                   |                     |                                                |
|                      |                   |                     | 고수준 래퍼.                                   |
|                      |                   |                     |                                                |
|                      |                   |                     | VBA 없이 Excel 완전 제어                       |
+----------------------+-------------------+---------------------+------------------------------------------------+
| **R + RDCOMClient**  | `RDCOMClient`     | **Windows 전용**    | R에서 Excel COM 객체를 직접 제어.              |
|                      |                   |                     |                                                |
|                      |                   |                     | VBA 대체 가능                                  |
+----------------------+-------------------+---------------------+------------------------------------------------+

이 방법들 중에서 Windows와 macOS에서 모두 가능한 방법인 Python + xlwings를 선택해서 아래의 실습을 진행하코자 합니다.

[1단계] xlwings 패키지 불러오기 
- `import` 명령은 원하는 파이썬 패키지를 현재 프로젝트 Python 환경에서 사용할 수 있도록 불러오는 명령입니다.      
- 이는 R에서의 `library("패키지명")`와 유사한 개념입니다. 
- 불러오기 위해서는 파이썬 가상환경에 해당 패키지가 미리 설치되어 있어야 합니다.
- `as xw`는 긴 패키지명을 짧은 별칭(alias)으로 사용하기 위한 관례적 표기법입니다.

In [21]:
import xlwings as xw

[2단계] 엑셀파일 열기
- `xw.Book(파일명)` 명령은 지정한 Excel 파일을 실제로 열고, 이를 제어할 수 있는 Workbook 객체를 생성합니다.   
- 생성된 객체는 변수 `wb_target`에 저장되어 이후 Excel 파일 조작에 사용됩니다.
- 파이썬에서는 `객체.속성` 또는 `객체.메소드()` 방식으로 해당 객체의 기능을 사용합니다.

In [22]:
wb_target = xw.Book('../data/deid/deid_KQIPS eCRF (수신-전산팀) 20250508 수정_수술전후검사결과제공_20250529.xlsx')

[3단계] 시트(sheet) 접근 방법   
아래 명령들은 Excel 통합문서(Workbook) 내의 시트들을 조작하는 기본적인 세 가지 접근 방식을 보여줍니다.   

1. **wb.sheets**   
    통합문서에 포함된 모든 시트의 컬렉션(collection) 객체입니다.   
    Python의 list처럼 인덱싱, 반복(iteration), 길이 확인(`len()`)이 가능합니다.   

2. **wb.sheets['시트명']**   
    시트 이름(name)으로 특정 시트를 선택합니다.   
    예: `wb.sheets['mapping']`

3. **wb.sheets[인덱스]**   
    시트 순서(index)로 접근합니다. (0부터 시작)   
    예: `wb.sheets[0]` → 첫 번째 시트

In [23]:
#모든 시트를 포함하는 컬랙션  
print(len(wb_target.sheets))

#시트명으로 조회
sheet_P_IC = wb_target.sheets['P_IC']
print(sheet_P_IC.name)

#시트순서로 조회  
sheet_1st = wb_target.sheets[0]
print(sheet_1st.name)

61
P_IC
P_IC


[4단계] 순회(Iteration)   
아래 예시는 통합문서의 모든 시트를 순회하며 번호와 이름을 출력하는 가장 기본적이며 직관적인 구조를 보여줍니다.

- **`for sheet in wb_target.sheets:`** : Python의 for문을 사용해 각 시트 객체를 차례로 참조합니다
- **`sheet.name`** : 현재 순회 중인 시트의 이름을 반환하는 속성입니다  
- **카운터 변수 `i`** : 시트의 순서를 표시하기 위해 수동으로 증가시키는 변수입니다
- **f-string** : `f"{i}. {sheet.name}"` 형식으로 번호와 시트명을 함께 출력합니다

In [24]:
i=0
for sheet in wb_target.sheets:
    i=i+1
    print(f"{i}. {sheet.name}")

1. P_IC
2. P_PRE_OP
3. P_PRE_OP;Q_SYMCATCO
4. P_PRE_OP;Q_SHABCATCO
5. P_PRE_OP;Q_SHNYNCO
6. P_PRE_OP;Q_MHETCYNCO
7. P_PRE_OP;Q_TFTRTCATCO
8. P_PRE_TEST
9. P_PRE_TEST;Q_LB
10. P_PRE_TEST;Q_LBE
11. P_PRE_TEST;Q_COLCATCO
12. P_PRE_TEST;Q_ANTRGCATCO
13. P_PRE_TEST;Q_ANIMMCATCO
14. P_OP
15. P_OP;Q_DXOPCATCO
16. P_OP;Q_TRTOCATCO
17. P_OP;Q_FDANCAT3
18. P_OP;Q_FDCOPCATCO
19. P_POST_TEST
20. P_POST_TEST;Q_LBA1
21. P_POST_TEST;Q_LBA2
22. P_POST_TEST;Q_LBA3
23. P_COMP
24. P_COMP;Q_CPR
25. P_COMP;Q_CPRC
26. P_COMP;Q_CPC
27. P_COMP;Q_CPCC
28. P_COMP;Q_CPL
29. P_COMP;Q_CPLC
30. P_COMP;Q_CPK
31. P_COMP;Q_CPKC
32. P_COMP;Q_CPM
33. P_COMP;Q_CPMC
34. P_COMP;Q_CPG
35. P_COMP;Q_CPGC
36. P_COMP;Q_CPI
37. P_COMP;Q_CPIC
38. P_COMP;Q_CPN
39. P_COMP;Q_CPNC
40. P_COMP;Q_CPP
41. P_COMP;Q_CPPC
42. P_COMP;Q_CPD
43. P_COMP;Q_CPDC
44. P_COMP;Q_CPO
45. P_COMP;Q_CPOC
46. P_COMP;Q_CPA
47. P_COMP;Q_CPAC
48. P_POST_CARE
49. P_POST_CARE;Q_ERCATCO
50. P_POST_CARE;Q_ERNPCATCO
51. P_POST_PATH
52. P_POST_PATH;Q_PATNM
53. P_P

[5단계] pandas import   
파이썬에서 **데이터프레임(DataFrame)**을 다루기 위해 pandas 패키지를 import 합니다.

- **pandas**: Python에서 가장 널리 사용되는 데이터 분석 라이브러리입니다
- **as pd**: 관례적으로 `pd`라는 짧은 별칭을 사용합니다
- **xlwings와의 연동**: xlwings는 Excel 데이터를 pandas DataFrame으로 쉽게 변환할 수 있습니다
- **주요 용도**: Excel에서 읽어온 데이터를 Python에서 분석, 조작, 처리하기 위해 사용됩니다

In [25]:
import pandas as pd

[6단계] 특정시트/특정영역을 데이터프레임으로 읽어오기   

Excel의 특정 시트에서 데이터 영역을 pandas DataFrame으로 변환하는 과정입니다.

- **`xw.Book(파일명)`**: 새로운 Excel 파일(mapping 파일)을 엽니다
- **`sheets['mapping']`**: 'mapping'이라는 이름의 시트를 선택합니다  
- **`range('A1:B1').expand('down')`**: A1:B1 범위에서 시작하여 데이터가 있는 마지막 행까지 자동 확장합니다
- **`options(pd.DataFrame, header=1, index=False)`**: xlwings 옵션 설정
  - `pd.DataFrame`: pandas DataFrame으로 변환
  - `header=1`: 첫 번째 행을 컬럼명으로 사용 
  - `index=False`: pandas 인덱스를 생성하지 않음
- **`.value`**: 실제 데이터 값을 반환하여 DataFrame 생성
- **`df.info()`**: 생성된 DataFrame의 기본 정보를 출력합니다

In [26]:
wb_mapping = xw.Book('../data/deid/deid_전체환자_등록번호_등재번호.xlsx')
sheet_mapping = wb_mapping.sheets['mapping']
df = sheet_mapping.range('A1:B1').expand('down').options(pd.DataFrame, header=1, index=False).value
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1857 entries, 0 to 1856
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pseudo_등록번호  1857 non-null   object
 1   등재번호         1857 non-null   object
dtypes: object(2)
memory usage: 29.1+ KB
None


[7단계] 컬럼내 중복행 찾기

DataFrame에서 중복된 값을 단계별로 찾아보는 과정입니다.
1. **중복 확인**: `duplicated()` 메소드로 각 행의 중복 여부를 Boolean으로 확인
2. **행 필터링**: Boolean 마스크를 사용해 중복된 행들만 추출  


In [27]:
# 단계별로 분해하여 직관적으로 작성
# 1단계: 행별로 중복여부를 Serires로 얻기 (=R에서의 vector와 유사)
is_duplicated = df['pseudo_등록번호'].duplicated()
print(type(is_duplicated))
print(is_duplicated)

# 2단계: 중복된 행들만 필터링 Boolin indexing
duplicated_rows = df[is_duplicated]
print("\n")
print(type(duplicated_rows))
print(duplicated_rows)

<class 'pandas.core.series.Series'>
0       False
1       False
2       False
3       False
4       False
        ...  
1852    False
1853    False
1854    False
1855    False
1856    False
Name: pseudo_등록번호, Length: 1857, dtype: bool


<class 'pandas.core.frame.DataFrame'>
     pseudo_등록번호          등재번호
1418    37303444  03A-00029992


[8단계] 엑셀에서 중복행 제거하기
pandas에서 찾은 중복 행을 실제 Excel 파일에서 삭제하는 과정입니다.
- **`sheet_mapping.api`**: xlwings의 저수준 Excel COM API에 접근
- **`.Rows(행번호).Delete()`**: 지정된 행 번호의 행을 Excel에서 직접 삭제
- **행 번호 주의**: Excel은 1부터 시작, pandas는 0부터 시작하므로 변환 필요


In [28]:
sheet_mapping.api.Rows(1419).Delete()

True

[9단계] 다른이름으로 엑셀 저장
- **`wb_mapping.save(새파일명)`**: 수정된 내용을 새 파일로 저장


In [29]:
wb_mapping.save('../data/replaced/중복제거_deid_전체환자_등록번호_등재번호.xlsx')

[10단계] 특정셀에 문자열 입력하기

xlwings를 사용하여 특정 Excel 셀에 값을 직접 입력하는 방법입니다.

**기본 문법**:
- **`sheet.range('셀주소').value = 값`**: 지정된 셀에 값을 입력
- **셀 주소**: Excel 형식 (예: 'A1', 'B8', 'C10')
- **입력 가능한 값**: 문자열, 숫자, 날짜, Boolean 등

**특징**:
- **즉시 반영**: 실행과 동시에 Excel 파일에 값이 입력됨
- **기존 값 덮어쓰기**: 해당 셀에 이미 값이 있어도 새 값으로 대체
- **서식 유지**: 셀의 기존 서식(폰트, 색상 등)은 유지되면서 값만 변경
- **자동 타입 변환**: Python의 데이터 타입이 Excel에 맞게 자동 변환

In [30]:
sheet_1st.range('B8').value = "문자열입력테스트"

[11단계] 특정셀에 함수로 입력하기

xlwings를 사용하여 Excel 셀에 수식(함수)을 직접 입력하는 방법입니다.

**기본 문법**:
- **`sheet.range('셀주소').formula = '=수식'`**: 지정된 셀에 Excel 수식을 입력
- **수식 형식**: 반드시 `=`로 시작하는 Excel 표준 수식 문법 사용
- **참조 방식**: 상대참조, 절대참조, 외부파일 참조 모두 가능

**VLOOKUP 함수 예시**:
- **`=VLOOKUP(찾을값, 범위, 열번호, 정확히찾기)`**
- **외부파일 참조**: `[파일명.xlsx]시트명!범위` 형식으로 다른 파일 참조
- **동적 셀 참조**: f-string을 사용해 행 번호를 변수로 처리

**특징**:
- **수식 계산**: Excel이 자동으로 수식을 계산하여 결과값 표시
- **참조 업데이트**: 참조된 데이터가 변경되면 자동으로 재계산

In [31]:
sheet_1st.range('B8').formula  = '=VLOOKUP(A8, [중복제거_deid_전체환자_등록번호_등재번호.xlsx]mapping!A:B, 2, FALSE)'

[12단계] 순회로 컬럼 전체에 적용하기

for 반복문을 사용하여 Excel의 특정 컬럼 전체에 동일한 패턴의 수식을 일괄 적용하는 방법입니다.

**핵심 개념**:
- **`range(시작행, 끝행+1)`**: Python의 range 함수로 행 번호 범위 생성
- **f-string 동적 참조**: `f"A{r}"`처럼 행 번호를 변수로 처리하여 각 행에 맞는 수식 생성

**실행 과정**:
1. **범위 설정**: `last_row = 1856`으로 마지막 행 번호 지정
2. **반복 실행**: 8행부터 1856행까지 각 행에 대해 수식 입력
3. **동적 수식**: 각 행마다 `A8`, `A9`, `A10`... 형태로 참조 셀이 자동 변경
4. **결과**: B8:B1856 범위의 모든 셀에 VLOOKUP 수식이 입력됨

In [32]:
last_row = 1856
for r in range(8, last_row + 1):
    formula = f"=VLOOKUP(A{r}, [중복제거_deid_전체환자_등록번호_등재번호.xlsx]mapping!A:B, 2, FALSE)"
    sheet_1st.range(f"B{r}").formula = formula


[13단계] 순회로 모든 시트에 적용하기

중첩된 for 반복문을 사용하여 Excel 통합문서의 **모든 시트**에 동일한 작업을 일괄 적용하는 고급 자동화 기법입니다.

**핵심 구조**:
```python
for sheet in wb_target.sheets:          # 외부 루프: 모든 시트 순회
    for r in range(8, last_row + 1):    # 내부 루프: 각 시트의 모든 행 순회
```

**동적 행 찾기**: `sheet.range('A:A').end('down').row`로 각 시트의 실제 마지막 행을 자동 감지

In [ ]:
# 모든 시트에 VLOOKUP 수식 일괄 적용 (오류 처리 포함)
i=0
for sheet in wb_target.sheets:
    i=i+1
    print(f"처리 중인 시트: {i}. '{sheet.name}'")
    if sheet.name == "P_POST_TEST":
        print(f"  - 건너뛰는 시트: '{sheet.name}'")
        continue  # 이 시트는 건너뛰고 다음 시트로

    
    # A열에서 "UPLOAD_ID"가 있는 행 찾기
    first_row = None
    for row in range(5, 8):  # 5-80행 내에서 검색
        cell_value = sheet.range(f'A{row}').value
        if cell_value and 'UPLOAD_ID' in str(cell_value):
            first_row = row
            break
    
    # A7부터 시작하여 아래로 내려가며 데이터가 있는 마지막 행 찾기
    last_row = sheet.range('A7').end('down').row
    
    print(f"  - UPLOAD_ID 위치: {first_row}행")
    print(f"  - A7부터 마지막 행: {last_row}행")
    
    # 안전한 방법: 개별 셀 처리 + 오류 처리
    if first_row:
        success_count = 0
        error_count = 0
        
        for r in range(first_row + 1, last_row + 1):
            try:
                cell = sheet.range(f"B{r}")
                
                # 먼저 셀을 초기화 (내용과 형식 모두 제거)
                cell.clear()
                
                # VLOOKUP 수식 입력
                formula = f"=VLOOKUP(A{r}, [중복제거_deid_전체환자_등록번호_등재번호.xlsx]mapping!A:B, 2, FALSE)"
                cell.formula = formula
                
                success_count += 1
                
            except Exception as e:
                print(f"    경고: B{r} 셀 처리 실패 - {str(e)[:50]}...")
                error_count += 1
                continue
        
        print(f"  - 완료: {success_count}개 셀 성공, {error_count}개 셀 실패")
    
    print()

처리 중인 시트: 'P_IC'
  - UPLOAD_ID 위치: 7행
  - A7부터 마지막 행: 1948행
  - 완료: 1941개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_OP'
  - UPLOAD_ID 위치: 7행
  - A7부터 마지막 행: 1948행
  - 완료: 1941개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_OP;Q_SYMCATCO'
  - UPLOAD_ID 위치: 6행
  - A7부터 마지막 행: 1947행
  - 완료: 1941개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_OP;Q_SHABCATCO'
  - UPLOAD_ID 위치: 6행
  - A7부터 마지막 행: 1947행
  - 완료: 1941개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_OP;Q_SHNYNCO'
  - UPLOAD_ID 위치: 6행
  - A7부터 마지막 행: 1947행
  - 완료: 1941개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_OP;Q_MHETCYNCO'
  - UPLOAD_ID 위치: 6행
  - A7부터 마지막 행: 1947행
  - 완료: 1941개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_OP;Q_TFTRTCATCO'
  - UPLOAD_ID 위치: 6행
  - A7부터 마지막 행: 1947행
  - 완료: 1941개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_TEST'
  - UPLOAD_ID 위치: 7행
  - A7부터 마지막 행: 1948행
  - 완료: 1941개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_TEST;Q_LB'
  - UPLOAD_ID 위치: 7행
  - A7부터 마지막 행: 48532행
  - 완료: 48525개 셀 성공, 0개 셀 실패

처리 중인 시트: 'P_PRE_TEST;Q_LBE'
  - UPLOAD_ID 위치: 7행
  - A7부터 마지막 행: 85행
  - 완료: 78개 셀 성공, 0개 셀 실패

처

[14단계] 완료된 엑셀 다른이름으로 저장하기

In [34]:
wb_target.save('../data/replaced/대체완료_deid_KQIPS eCRF (수신-전산팀) 20250508 수정_수술전후검사결과제공_20250529.xlsx')

In [35]:
wb_target.close()


In [36]:
wb_validation = xw.Book('../data/replaced/대체완료_deid_KQIPS eCRF (수신-전산팀) 20250508 수정_수술전후검사결과제공_20250529.xlsx')

i=0
for sheet in wb_validation.sheets:
    i=i+1
    print(f"{i}. {sheet.name}")
    
    # A열에서 "UPLOAD_ID"가 있는 행 찾기
    first_row = None
    for row in range(1, 100):  # 처음 100행 내에서 검색
        cell_value = sheet.range(f'B{row}').value
        if cell_value and 'SUBJID' in str(cell_value):
            first_row = row
            break
    
    # A7부터 시작하여 아래로 내려가며 데이터가 있는 마지막 행 찾기
    last_row = sheet.range('B7').end('down').row
    
    print(f"  - SUBJID 위치: {first_row}행")
    print(f"  - B7부터 마지막 행: {last_row}행")
    
    
    print()    


1. P_IC
  - SUBJID 위치: 7행
  - B7부터 마지막 행: 1948행

2. P_PRE_OP
  - SUBJID 위치: 7행
  - B7부터 마지막 행: 1948행

3. P_PRE_OP;Q_SYMCATCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

4. P_PRE_OP;Q_SHABCATCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

5. P_PRE_OP;Q_SHNYNCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

6. P_PRE_OP;Q_MHETCYNCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

7. P_PRE_OP;Q_TFTRTCATCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

8. P_PRE_TEST
  - SUBJID 위치: 7행
  - B7부터 마지막 행: 1948행

9. P_PRE_TEST;Q_LB
  - SUBJID 위치: 7행
  - B7부터 마지막 행: 48532행

10. P_PRE_TEST;Q_LBE
  - SUBJID 위치: 7행
  - B7부터 마지막 행: 85행

11. P_PRE_TEST;Q_COLCATCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

12. P_PRE_TEST;Q_ANTRGCATCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

13. P_PRE_TEST;Q_ANIMMCATCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

14. P_OP
  - SUBJID 위치: 7행
  - B7부터 마지막 행: 1948행

15. P_OP;Q_DXOPCATCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

16. P_OP;Q_TRTOCATCO
  - SUBJID 위치: 6행
  - B7부터 마지막 행: 1947행

17. P_OP;Q_FDANCAT3
